In [74]:
import numpy as np 
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder, MinMaxScaler , OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline 
from sklearn.preprocessing import PowerTransformer
from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer

In [75]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
warnings.filterwarnings('ignore')
%matplotlib inline

In [76]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [77]:
train = pd.read_csv('train.csv').drop('Id',axis = 1 )
test = pd.read_csv('test.csv').drop('Id',axis = 1 )


In [78]:
train.drop(train[(train['OverallQual']<5) & (train['SalePrice']>200000)].index, inplace=True)
train.drop(train[(train['GrLivArea']>4500) & (train['SalePrice']<300000)].index, inplace=True)
train.reset_index(drop=True, inplace=True)

train["SalePrice"] = np.log1p(train["SalePrice"])

In [79]:
train_labels = train['SalePrice'].reset_index(drop=True)
train_features = train.drop(['SalePrice'], axis=1)
test_features = test

# Объединяем тестовую и тренировочную выборку
all_features = pd.concat([train_features, test_features]).reset_index(drop=True)
all_features.shape

(2916, 79)

In [80]:
def percent_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().mean()*100,2)})
    
    return dict_x

missing = percent_missing(all_features)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:10]

Percent of missing data


[('PoolQC', 99.69),
 ('MiscFeature', 96.4),
 ('Alley', 93.21),
 ('Fence', 80.42),
 ('FireplaceQu', 48.7),
 ('LotFrontage', 16.63),
 ('GarageYrBlt', 5.45),
 ('GarageFinish', 5.45),
 ('GarageQual', 5.45),
 ('GarageCond', 5.45)]

In [81]:
all_features['MSSubClass'] = all_features['MSSubClass'].apply(str)
all_features['YrSold'] = all_features['YrSold'].astype(str)
all_features['MoSold'] = all_features['MoSold'].astype(str)
all_features['YearRemodAdd'] = all_features['YearRemodAdd'].astype(str)
all_features['YearBuilt'] = all_features['YearBuilt'].astype(str)

In [82]:
def handle_missing(features):
    # the data description states that NA refers to typical ('Typ') values
    features['Functional'] = features['Functional'].fillna('Typ')
    # Replace the missing values in each of the columns below with their mode
    features['Electrical'] = features['Electrical'].fillna("SBrkr")
    features['KitchenQual'] = features['KitchenQual'].fillna("TA")
    features['Exterior1st'] = features['Exterior1st'].fillna(features['Exterior1st'].mode()[0])
    features['Exterior2nd'] = features['Exterior2nd'].fillna(features['Exterior2nd'].mode()[0])
    features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])
    features['MSZoning'] = features.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))
    
    features["PoolQC"] = features["PoolQC"].fillna("None")
    for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
        features[col] = features[col].fillna(0)
    for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
        features[col] = features[col].fillna('None')
    for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
        features[col] = features[col].fillna('None')
        
    features['LotFrontage'] = features.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    objects = []
    for i in features.columns:
        if features[i].dtype == object:
            objects.append(i)
    features.update(features[objects].fillna('None'))
        
    numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numeric = []
    for i in features.columns:
        if features[i].dtype in numeric_dtypes:
            numeric.append(i)
    features.update(features[numeric].fillna(0))    
    return features

all_features = handle_missing(all_features)

In [83]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = []
for i in all_features.columns:
    if all_features[i].dtype in numeric_dtypes:
        numeric.append(i)

In [84]:
skew_features = all_features[numeric].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(10)

There are 25 numerical features with Skew > 0.5 :


MiscVal          21.935910
PoolArea         17.685603
LotArea          13.262550
LowQualFinSF     12.082427
3SsnPorch        11.370087
KitchenAbvGr      4.299698
BsmtFinSF2        4.143683
EnclosedPorch     4.001570
ScreenPorch       3.944305
BsmtHalfBath      3.929197
dtype: float64

In [85]:
mmx = MinMaxScaler()

In [86]:
# Normalize skewed features
for i in skew_index:
    all_features[i] = boxcox1p(all_features[i], boxcox_normmax(all_features[i] + 1))

In [87]:
for col in all_features.columns[all_features.dtypes != 'O']:
    all_features[col] = mmx.fit_transform(all_features[col].values.reshape(-1,1))

In [88]:
all_features['MSSubClass'] = all_features['MSSubClass'].apply(float)
all_features['YrSold'] = all_features['YrSold'].astype(int)
all_features['MoSold'] = all_features['MoSold'].astype(int)
all_features['YearRemodAdd'] = all_features['YearRemodAdd'].astype(int)
all_features['YearBuilt'] = all_features['YearBuilt'].astype(int)

In [89]:
all_features = pd.get_dummies(all_features).reset_index(drop=True)
all_features.shape

(2916, 302)

In [90]:
all_features = all_features.loc[:,~all_features.columns.duplicated()]

In [91]:
X= all_features.iloc[:len(train_labels), :]
XX = all_features.iloc[len(train_labels):, :]
X.shape, train_labels.shape, XX.shape

((1457, 302), (1457,), (1459, 302))

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X,train_labels, test_size=0.33, random_state=42)

In [116]:
LR = LinearRegression()
LR.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [117]:
y_pred = LR.predict(X_test)

In [118]:
print(f'MAE: {metrics.mean_absolute_error(y_test,y_pred)}')
print(f'MSE: {metrics.mean_squared_error(y_test,y_pred)}')
print(f'RMSE: {np.sqrt(metrics.mean_squared_error(y_test,y_pred))}')
print(f'R2 score: {metrics.r2_score(y_test,y_pred)}')


MAE: 0.10239943049918261
MSE: 0.0810466783032851
RMSE: 0.28468698302396106
R2 score: 0.45662866161999716


In [121]:
RF = RandomForestRegressor(n_estimators=100)
RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)
print("MAE:", metrics.mean_absolute_error(y_test, y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2',RF.score(X_test,y_test))

MAE: 0.09438898672213702
MSE: 0.01977481547800879
RMSE: 0.14062295501805097
R2 0.867421241962637


In [119]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

feature_sel_model = SelectFromModel(Lasso(alpha=0.005, random_state=0)) 
feature_sel_model.fit(X_train, y_train)
selected_feat = X_train.columns[(feature_sel_model.get_support())]
X_train_lasso = X_train[selected_feat].reset_index(drop=True)
X_test_lasso =X_test[selected_feat]


In [120]:
LassoLR = LinearRegression().fit(X_train_lasso,y_train)
y_pred_lasso = LassoLR.predict(X_test_lasso)
print("MAE:", metrics.mean_absolute_error(y_test, y_pred_lasso))
print('MSE:', metrics.mean_squared_error(y_test, y_pred_lasso))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_lasso)))
print('R2',LassoLR.score(X_test_lasso,y_test))


MAE: 0.09563063201455059
MSE: 0.01872941158972384
RMSE: 0.13685544048273654
R2 0.8744300734387325
